In [1]:
# Importando bibliotecas
from pyspark.sql import SparkSession

# Criando objeto de sessão
spark = SparkSession\
    .builder\
    .appName("tipos-primitivos-complexos")\
    .master("local[1]")\
    .getOrCreate()

spark

22/09/01 09:38:10 WARN Utils: Your hostname, panini-ubuntu resolves to a loopback address: 127.0.1.1, but we couldn't find any external IP address!
22/09/01 09:38:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/01 09:38:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/01 09:38:12 WARN MacAddressUtil: Failed to find a usable hardware address from the network interfaces; using random bytes: e8:3b:9f:5f:1f:63:47:3a


In [8]:
# Importando tipos primitivos
from pyspark.sql.types import StructType, StructField,\
    IntegerType, StringType, MapType, ArrayType

# Definindo schema
schema = StructType([
    StructField("time", StringType(), nullable=False),
    StructField("qtd_titulos_br", IntegerType(), nullable=True),
    StructField("anos_titulos_br", ArrayType(IntegerType()), nullable=True),
    StructField("estadio", MapType(StringType(), StringType()), nullable=True)
])

# Criando dados manualmente (python)
data_times_br = [
    ["Corinthians", 7,  [1990, 1998, 1999, 2005, 2011, 2015, 2017], {"nome": "Neo Quimica Arena", "regiao": "Itaquera", "linha_metro": "Vermelha"}],
    ["Palmeiras", 10, [1960, 1967, 1967, 1969, 1972, 1973, 1993, 1994, 2016, 2018], {"nome": "Allianz Park", "regiao": "Barra Funda", "linha_metro": "Vermelha"}],
    ["São Paulo", 6, [1977, 1986, 1991, 2006, 2007, 2008], {"nome": "Morumbi", "regiao": "Morumbi", "linha_metro": "Amarela"}],
    ["Santos", 8, [1961, 1962, 1963, 1964, 1965, 1968, 2002, 2004], {"nome": "Vila Belmiro", "regiao": "Baixada Santista"}]
]

# Transformando dados em DataFrame
df_times_br = spark.createDataFrame(data_times_br, schema)

# Visualizando dados
df_times_br.printSchema()
df_times_br.show(truncate=True)

root
 |-- time: string (nullable = false)
 |-- qtd_titulos_br: integer (nullable = true)
 |-- anos_titulos_br: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- estadio: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+-----------+--------------+--------------------+--------------------+
|       time|qtd_titulos_br|     anos_titulos_br|             estadio|
+-----------+--------------+--------------------+--------------------+
|Corinthians|             7|[1990, 1998, 1999...|{linha_metro -> V...|
|  Palmeiras|            10|[1960, 1967, 1967...|{linha_metro -> V...|
|  São Paulo|             6|[1977, 1986, 1991...|{linha_metro -> A...|
|     Santos|             8|[1961, 1962, 1963...|{nome -> Vila Bel...|
+-----------+--------------+--------------------+--------------------+



## Operações em tipos primitivos complexos

### Selecionando Tipos Complexos

In [9]:
# Importando funções
from pyspark.sql.functions import col, expr

# Visualizando primeiro e segundo títulos brasileiro
df_titulos_br = df_times_br.select(
    expr("time"),
    col("anos_titulos_br")[0].alias("ano_primeiro_titulo"),
    expr("anos_titulos_br[1] AS ano_segundo_titulo")
)

# Visualizando dados
df_titulos_br.show()

+-----------+-------------------+------------------+
|       time|ano_primeiro_titulo|ano_segundo_titulo|
+-----------+-------------------+------------------+
|Corinthians|               1990|              1998|
|  Palmeiras|               1960|              1967|
|  São Paulo|               1977|              1986|
|     Santos|               1961|              1962|
+-----------+-------------------+------------------+



In [10]:
# Visualizando detalhes dos estádios dos times
df_estadios = df_times_br.select(
    col("time"),
    col("estadio").nome,
    col("estadio").regiao.alias("regiao_estadio"),
    expr("estadio.linha_metro AS metro_prox_estadio")
)

# Visualizando dados
df_estadios.show()

+-----------+-----------------+----------------+------------------+
|       time|    estadio[nome]|  regiao_estadio|metro_prox_estadio|
+-----------+-----------------+----------------+------------------+
|Corinthians|Neo Quimica Arena|        Itaquera|          Vermelha|
|  Palmeiras|     Allianz Park|     Barra Funda|          Vermelha|
|  São Paulo|          Morumbi|         Morumbi|           Amarela|
|     Santos|     Vila Belmiro|Baixada Santista|              null|
+-----------+-----------------+----------------+------------------+



___

### element_at

In [13]:
# Importando função
from pyspark.sql.functions import element_at

# Visualizando primeiro e último título brasileiro
df_titulos_br = df_times_br.select(
    col("time"),
    element_at(col("anos_titulos_br"), 1).alias("ano_primeiro_titulo"),
    element_at(col("anos_titulos_br"), -1).alias("ano_ultimo_titulo"),
    expr("year(current_date()) - element_at(anos_titulos_br, -1) AS anos_sem_titulos"),
)

# Visualizando dados
df_titulos_br.show()

+-----------+-------------------+-----------------+----------------+
|       time|ano_primeiro_titulo|ano_ultimo_titulo|anos_sem_titulos|
+-----------+-------------------+-----------------+----------------+
|Corinthians|               1990|             2017|               5|
|  Palmeiras|               1960|             2018|               4|
|  São Paulo|               1977|             2008|              14|
|     Santos|               1961|             2004|              18|
+-----------+-------------------+-----------------+----------------+



### slice

In [20]:
# Importando função
from pyspark.sql.functions import slice

# Selecionando primeiros títulos
df_prim_titulos = df_times_br.select(
    col("time"),
    slice(col("anos_titulos_br"), 1, 3).alias("primeiros_titulos"),
)

# Visualizando dados
df_prim_titulos.show()

+-----------+------------------+
|       time| primeiros_titulos|
+-----------+------------------+
|Corinthians|[1990, 1998, 1999]|
|  Palmeiras|[1960, 1967, 1967]|
|  São Paulo|[1977, 1986, 1991]|
|     Santos|[1961, 1962, 1963]|
+-----------+------------------+



### size

In [28]:
# Importando função
from pyspark.sql.functions import size

# Aplicando consulta sobre títulos br
df_titulos = df_times_br.select(
    col("time"),
    col("qtd_titulos_br"),
    size(col("anos_titulos_br")).alias("qtd_titulos_br_size"),
    col("anos_titulos_br")[size(col("anos_titulos_br")) - 1].alias("ano_ultimo_titulo"),
    expr("size(estadio) AS qtd_infos_estadio")
)

# Visualizando dados
df_titulos.show()

+-----------+--------------+-------------------+-----------------+-----------------+
|       time|qtd_titulos_br|qtd_titulos_br_size|ano_ultimo_titulo|qtd_infos_estadio|
+-----------+--------------+-------------------+-----------------+-----------------+
|Corinthians|             7|                  7|             2017|                3|
|  Palmeiras|            10|                 10|             2018|                3|
|  São Paulo|             6|                  6|             2008|                3|
|     Santos|             8|                  8|             2004|                2|
+-----------+--------------+-------------------+-----------------+-----------------+



### array_contains

In [30]:
# Importando função
from pyspark.sql.functions import array_contains

# Analisando títulos em ano específico
df_campeoes = df_times_br.select(
    col("time"),
    array_contains(col("anos_titulos_br"), 2015).alias("campeao_2015"),
    expr("array_contains(anos_titulos_br, 2018) AS campeao_2018")
)

# Visualizando dados
df_campeoes.show()

+-----------+------------+------------+
|       time|campeao_2015|campeao_2018|
+-----------+------------+------------+
|Corinthians|        true|       false|
|  Palmeiras|       false|        true|
|  São Paulo|       false|       false|
|     Santos|       false|       false|
+-----------+------------+------------+



### explode

In [35]:
# Importando função
from pyspark.sql.functions import explode

# Analisando todos os anos de títulos dos times paulistas
df_conquistas_sp = df_times_br.select(
    explode(col("anos_titulos_br")).alias("conquistas_times_paulistas")
)

# Visualizando resultados
df_conquistas_sp.show()

+--------------------------+
|conquistas_times_paulistas|
+--------------------------+
|                      1990|
|                      1998|
|                      1999|
|                      2005|
|                      2011|
|                      2015|
|                      2017|
|                      1960|
|                      1967|
|                      1967|
|                      1969|
|                      1972|
|                      1973|
|                      1993|
|                      1994|
|                      2016|
|                      2018|
|                      1977|
|                      1986|
|                      1991|
+--------------------------+
only showing top 20 rows



In [39]:
# Analisando informações de estádios
df_info_estadios = df_times_br.select(
    col("time"),
    expr("explode(estadio)")
)

# Visualizando dados
df_info_estadios.show()

+-----------+-----------+-----------------+
|       time|        key|            value|
+-----------+-----------+-----------------+
|Corinthians|linha_metro|         Vermelha|
|Corinthians|       nome|Neo Quimica Arena|
|Corinthians|     regiao|         Itaquera|
|  Palmeiras|linha_metro|         Vermelha|
|  Palmeiras|       nome|     Allianz Park|
|  Palmeiras|     regiao|      Barra Funda|
|  São Paulo|linha_metro|          Amarela|
|  São Paulo|       nome|          Morumbi|
|  São Paulo|     regiao|          Morumbi|
|     Santos|       nome|     Vila Belmiro|
|     Santos|     regiao| Baixada Santista|
+-----------+-----------+-----------------+



### split

In [43]:
# Importando função
from pyspark.sql.functions import split

# Separando nomes de times
df_times = df_times_br.select(
    split(col("time"), " ")
)

# Visualizando dados
df_times.show()

+------------------+
|split(time,  , -1)|
+------------------+
|     [Corinthians]|
|       [Palmeiras]|
|      [São, Paulo]|
|          [Santos]|
+------------------+



In [54]:
# Analisando primeiro termo que nomeia os estadios
df_estadio_nome = df_times_br.selectExpr(
    "time",
    "explode(estadio) AS (chave, valor)"
).where(expr("chave = 'nome'")).selectExpr(
    "time",
    "element_at(split(valor, ' '), 1) AS primeiro_nome_estadio"
)

# Visualizando dados
df_estadio_nome.show()

+-----------+---------------------+
|       time|primeiro_nome_estadio|
+-----------+---------------------+
|Corinthians|                  Neo|
|  Palmeiras|              Allianz|
|  São Paulo|              Morumbi|
|     Santos|                 Vila|
+-----------+---------------------+



### map

In [60]:
# Importando função
from pyspark.sql.functions import create_map

# Gerando mappings
df_complex_map = df_times_br.select(
    create_map(col("time"), col("qtd_titulos_br")).alias("times_titulos")
)

# Visualizando dados
df_complex_map.printSchema()
df_complex_map.show()

root
 |-- times_titulos: map (nullable = false)
 |    |-- key: string
 |    |-- value: integer (valueContainsNull = true)

+------------------+
|     times_titulos|
+------------------+
|{Corinthians -> 7}|
| {Palmeiras -> 10}|
|  {São Paulo -> 6}|
|     {Santos -> 8}|
+------------------+



In [82]:
# Criando map aninhado
df_times_estadio = df_times_br.select(
    create_map(col("time"), col("estadio")).alias("time_info_estadio")
)

# Visualizando dados
df_times_estadio.printSchema()
df_times_estadio.show(truncate=False)

root
 |-- time_info_estadio: map (nullable = false)
 |    |-- key: string
 |    |-- value: map (valueContainsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)

+-----------------------------------------------------------------------------------------+
|time_info_estadio                                                                        |
+-----------------------------------------------------------------------------------------+
|{Corinthians -> {linha_metro -> Vermelha, nome -> Neo Quimica Arena, regiao -> Itaquera}}|
|{Palmeiras -> {linha_metro -> Vermelha, nome -> Allianz Park, regiao -> Barra Funda}}    |
|{São Paulo -> {linha_metro -> Amarela, nome -> Morumbi, regiao -> Morumbi}}              |
|{Santos -> {nome -> Vila Belmiro, regiao -> Baixada Santista}}                           |
+-----------------------------------------------------------------------------------------+



In [81]:
# Gerando dados do tipo struct
df_times_br.selectExpr(
    "(time, qtd_titulos_br) AS times_titulos_struct",
    "(time, qtd_titulos_br).time AS time",
    "(time, qtd_titulos_br).qtd_titulos_br AS qtd_titulos_br"
).show()

+--------------------+-----------+--------------+
|times_titulos_struct|       time|qtd_titulos_br|
+--------------------+-----------+--------------+
|    {Corinthians, 7}|Corinthians|             7|
|     {Palmeiras, 10}|  Palmeiras|            10|
|      {São Paulo, 6}|  São Paulo|             6|
|         {Santos, 8}|     Santos|             8|
+--------------------+-----------+--------------+

